In [ ]:
#import relevant libraries
import pandas as pd
from pandas import DataFrame,Series
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import os

In [ ]:
#Setting the current working directory
os.chdir(r'C:\Users\Cora Ryan\Downloads\Cross-Quantilogram-master')

In [ ]:
#importing packages from Cross-Quantilogram folder
from crossquantilogram import CrossQuantilogram as cq
from stationarybootstrap import Bootstrap
from utils import DescriptiveStatistics
from qtests import LjungBoxQ
from calculate import CQBS,CQBS_alphas,CQBS_years
#from plot import heatmap_example,rolling_example

In [ ]:
#Defining plotting function for cross-quantilograms

def plot_cqs(data_1,data_2,lags=30,alpha=0.1):
    fig,ax = plt.subplots(1,len(data_2.columns),figsize=(20,9))

    for i in range(len(data_2.columns)):
        dataset = CQBS(data1 = data_1,a1=alpha,data2 = data_2[data_2.columns[i]],a2=alpha,verbose=True,k=30)


        xaxis=[str(x) for x in dataset.index]
        ax[0].set_ylabel("Cross-Quantilogram",fontsize="xx-large",labelpad=0.1)
        ax[i].set_xlabel("Lag",fontsize="x-large",labelpad=0.1)
        ax[i].set_title(data_2.columns[i],fontsize="xx-large")       
        ax[i].tick_params(labelsize="large")
        ax[i].axhline(color="black",linewidth=1)
        ax[i].bar(xaxis,dataset["cq"],width=0.2,color="blue")
        ax[i].plot(xaxis,dataset["cq_upper"],color='red',linestyle="dashed")
        ax[i].plot(xaxis,dataset["cq_lower"],color='red',linestyle="dashed")
        ax[i].xaxis.set_major_locator(ticker.MultipleLocator(2))
        m = (abs(dataset["cq"]).max()//0.05)*0.05+0.1
        ax[i].set_ylim(-m,m)

    fig.savefig(data_1.name + "_cq_bar.png",dpi=200,quality=95,bbox_inches="tight")
    plt.show()
    
def heatmap(data1,data2,picname="",show=True,k=1):
    alist = [0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4]
    
    dataset = CQBS_alphas(data1,alist,data2,alist,k,verbose=True)
    data_mat=np.array([[d["cq"] for d in row] for row in dataset])
    data_txt=[["*" if ((d["cq"]>d["cq_upper"] or d["cq"]<d["cq_lower"])and d["q"]>d["qc"])\
                else "" for d in row] for row in dataset]

    fig, ax = plt.subplots(figsize=(4,5))
    im = ax.imshow(data_mat, cmap="RdBu_r",vmin=-0.1,vmax=0.1)
    ax.set_ylabel(data1.name,fontsize="xx-large",verticalalignment="center",labelpad=10)
    ax.set_xlabel(data2.name,fontsize="xx-large",labelpad=35)
    cbar = ax.figure.colorbar(im,ax=ax,fraction=0.046,pad=0.02,orientation="horizontal",)
    cbar.ax.tick_params(labelsize="large")
    cbar.ax.set_ylabel("", rotation=-90, va="bottom")
    ax.set_xticks(np.arange(data_mat.shape[1]))
    ax.set_yticks(np.arange(data_mat.shape[0]))
    ax.set_yticklabels(alist,horizontalalignment="right",fontsize="large")
    ax.set_xticklabels(alist,fontsize="large")    
    ax.tick_params(top=True, bottom=False,labeltop=True, labelbottom=False)                   
    plt.setp(ax.get_xticklabels(), rotation=-60, ha="right",rotation_mode="anchor")        
    ax.set_xticks(np.arange(data_mat.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data_mat.shape[0]+1)-.5, minor=True)
    ax.tick_params(which="minor", bottom=False, left=False)

    data = im.get_array()
    valfmt="{x}"
    threshold = im.norm(data.max())/2.0
    textcolors=["black", "white"]
    kw = dict(horizontalalignment="center",verticalalignment="center")              
    valfmt = ticker.StrMethodFormatter(valfmt)

    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[im.norm(data[i, j]) > threshold])
            text = im.axes.text(j, i, valfmt(data_txt[i][j], None),fontsize="large",**kw)

    fig.tight_layout()
    if picname:
        fig.savefig(picname+".png",dpi=200,quality=95,bbox_inches="tight")
    if show:
        print(picname+":")
        plt.show()
        
def rolling_window(data1,data_2,alpha=0.1,picname="",show=True,k=1):
    fig,ax = plt.subplots(1,len(data_2.columns),figsize=(20,8))

    for i in range(len(data_2.columns)):
        dataset = CQBS_years(data1,alpha,data_2[data_2.columns[i]],alpha,verbose=True,k=k)
        plt.subplots_adjust(wspace=0.3,hspace=0.5) 
        xaxis=[x for x in dataset.index]

        ax[0].set_ylabel("Rolling\nCross-Quantilogram",fontsize="x-large",labelpad=0.1)
        ax[i].set_title(data_2.columns[i],fontsize="xx-large")     
        ax[i].tick_params(labelsize="large")
        ax[i].axhline(color="black",linewidth=1)        
        ax[i].plot(xaxis,dataset["cq_upper"],"rd--",markersize=5,markerfacecolor="w")
        ax[i].plot(xaxis,dataset["cq_lower"],"rd--",markersize=5,markerfacecolor="w")
        ax[i].plot(xaxis,dataset["cq"],"bo-",markersize=4)
        ax[i].xaxis.set_major_locator(ticker.MultipleLocator(2))
        m = (max(abs(dataset["cq"]).max(),abs(dataset["cq_upper"]).max(),abs(dataset["cq_lower"]).max())//0.05)*0.05+0.1
        ax[i].set_ylim(-m,m)

    fig.align_labels()

    if picname:
        fig.savefig(picname+".png",dpi=200,quality=95,bbox_inches="tight")   
    if show:
        print(picname+":")
        plt.show()

In [ ]:
#Reading in the data
#Cryptocurrencies
btc = pd.read_csv("BTC-USD.csv")
eth = pd.read_csv("ETH-USD.csv")
xrp = pd.read_csv("XRP-USD.csv")
ltc = pd.read_csv("LTC-USD.csv")
xlm = pd.read_csv("XLM-USD.csv")
xmr = pd.read_csv("XMR-USD.csv")
dash = pd.read_csv("DASH-USD.csv")
xem = pd.read_csv("XEM-USD.csv")

#Indices
ftse = pd.read_csv("FTSE 100 Historical Data.csv")
#changing the price datatype from object to float
ftse["Price"] = ftse["Price"].str.replace(",","").astype("float")

nkke = pd.read_csv("^N225.csv")
vix = pd.read_csv("^VIX.csv")
sp500 = pd.read_csv("^GSPC.csv")

In [ ]:
#Calculating log returns for each dataset (yahoo data)
yahoo_data = [btc,eth,xrp,nkke,vix,sp500,ltc,xem,xlm,xmr,dash]
yahoo_strs = ["BTC","ETH","XRP","Nikkei","VIX","SP500","Litecoin","NEM","Stellar","Monero","Dash"]
for i in range(len(yahoo_data)):
    yahoo_data[i].drop(columns=yahoo_data[i].columns[[1,2,3,4,-1]],inplace=True)
    yahoo_data[i]["Date"] = pd.to_datetime(yahoo_data[i]["Date"],format="%Y%m%d",infer_datetime_format=True)
    yahoo_data[i].set_index('Date',inplace=True)
    yahoo_data[i][yahoo_strs[i]] = 100*np.log(yahoo_data[i]["Adj Close"]).diff()
    yahoo_data[i].dropna(inplace=True)
    yahoo_data[i].drop("Adj Close",axis=1,inplace=True)

In [ ]:
#Calculating log returns for each dataset (investing.com data)
inv_com_data = [ftse]
inv_com_strs = ["FTSE 100"]
for i in range(len(inv_com_data)):
    inv_com_data[i].drop(columns=inv_com_data[i].columns[2:],inplace=True)
    inv_com_data[i]["Date"] = pd.to_datetime(inv_com_data[i]["Date"],format="%Y%m%d",infer_datetime_format=True)
    inv_com_data[i].set_index('Date',inplace=True)
    inv_com_data[i][inv_com_strs[i]] = 100*np.log(inv_com_data[i]["Price"]).diff()
    inv_com_data[i].dropna(inplace=True)
    inv_com_data[i].drop("Price",axis=1,inplace=True)

In [ ]:
all_df = pd.concat([btc,eth,xrp,vix,dash,xem,xlm,xmr,ltc,sp500,ftse,nkke],axis=1)
all_df.dropna(inplace=True)
all_df.columns

In [ ]:
safe_df = all_df.loc[:,["BTC","ETH","XRP","Litecoin","NEM","Stellar","Monero","Dash"]].copy()
safe_df.head()

In [ ]:
index_df = all_df.loc[:,["SP500","FTSE 100","Nikkei"]].copy()
index_df.head()

In [ ]:
#Getting Descriptive Statistics for variables

ovr_stats_dicts = []
for i in all_df.columns:
    x = DescriptiveStatistics(all_df[i],10)
    ovr_stats_dicts.append(x)

ovr_des_stats = []
for i in ovr_stats_dicts:
    x = []
    for key in i.keys():
        x.append(i[key])
    ovr_des_stats.append(x)

#create a dataframe for descriptive stats (index)
ovr_stats_df = DataFrame(ovr_des_stats,index=all_df.columns,columns=i.keys())
ovr_stats_df.adfs = ovr_stats_df.adfs.round(2)
#ovr_stats_df.to_excel("Overall Descriptive Statistics.xlsx")

In [ ]:
#Plotting Heatmaps of CQs
lags = [1,5,10]
for lag in lags:
    for i in safe_df.columns:
        for j in index_df.columns:
            heatmap(safe_df[i],index_df[j],picname=("Heatmap " + i + "_" + j + " k = " + str(lag)),k=lag)

In [ ]:
#Plotting the cross-quantilograms for each potential "safe-asset" against our indices
lag = 1
for i in safe_df.columns:
    rolling_window(safe_df[i],index_df,k=lag,picname=("Rolling Window " + i  + " k = " + str(lag)))